In [1]:
import pandas as pd
import numpy as np

import nltk
#from bs4 import BeautifulSoup

import gensim
from gensim import utils
import sys
from sklearn.datasets import fetch_20newsgroups
from nltk import word_tokenize
from nltk import download
from nltk.corpus import stopwords
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
from gensim.models.keyedvectors import KeyedVectors

model = KeyedVectors.load_word2vec_format('./words/GoogleNews-vectors-negative300.bin.gz', binary=True)
#model.save_word2vec_format('./words/GoogleNews-vectors-negative300.txt', binary=False)

In [3]:
download('punkt') #tokenizer, run once
download('stopwords') #stopwords dictionary, run once
stop_words = stopwords.words('english')

[nltk_data] Error loading punkt: <urlopen error [Errno 8] nodename nor
[nltk_data]     servname provided, or not known>
[nltk_data] Error loading stopwords: <urlopen error [Errno 8] nodename
[nltk_data]     nor servname provided, or not known>


In [4]:
# Excel on Mac saves csv file with delimiter ','
# Because there are many comma in texts, you should change them into other characters like space ' '
# I changed comma ',' in text to space' '
photo_posts = pd.read_csv('cs_pytesseract.csv')

In [5]:
photo_posts['manual'].isnull().any()

False

In [6]:
null_mask = photo_posts['manual'].isnull()

idx = np.where(null_mask)
idx
#photo_posts[null_mask]['message']
for i in idx:
    photo_posts.loc[i, 'manual'] = ''



#In [174]: mask = x[0]>0
#In [175]: mask
#Out[175]: array([False,  True, False,  True, False], dtype=bool)
##In [176]: idx = np.where(mask)[0]
#In [177]: idx
#Out[177]: array([1, 3], dtype=int32)
#In [178]: x[0, idx[1]]
#Out[178]: 6.0

photo_posts['manual'].isnull().any()

False

In [7]:
photo_posts['ocr'].isnull().any()

True

In [8]:
null_mask = photo_posts['ocr'].isnull()

idx = np.where(null_mask)
idx
#photo_posts[null_mask]['message']
for i in idx:
    photo_posts.loc[i, 'ocr'] = ''



#In [174]: mask = x[0]>0
#In [175]: mask
#Out[175]: array([False,  True, False,  True, False], dtype=bool)
##In [176]: idx = np.where(mask)[0]
#In [177]: idx
#Out[177]: array([1, 3], dtype=int32)
#In [178]: x[0, idx[1]]
#Out[178]: 6.0

photo_posts['ocr'].isnull().any()

False

In [16]:
manuals = photo_posts['manual'].tolist()
ocrs = photo_posts['ocr'].tolist()

In [17]:
def preprocess(text):
    text = text.lower()
    doc = word_tokenize(text)
    doc = [word for word in doc if word not in stop_words]
    doc = [word for word in doc if word.isalpha()] #restricts string to alphabetic characters only
    return doc

In [18]:
len(manuals)

164

In [19]:
t_corpus = [preprocess(str(text)) for text in manuals]

In [20]:
m_corpus = [preprocess(str(ocr)) for ocr in ocrs]
len(m_corpus)

164

In [24]:
def filter_docs(corpus, texts, condition_on_doc):
    """
    Filter corpus, texts and labels given the function condition_on_doc which takes
    a doc.
    The document doc is kept if condition_on_doc(doc) is true.
    """
    number_of_docs = len(corpus)

    #if texts is not None:
    #    texts = [text for (text, doc) in zip(texts, corpus)
    #             if condition_on_doc(doc)]

    #labels = [i for (i, doc) in zip(labels, corpus) if condition_on_doc(doc)]
    #corpus = [doc for doc in corpus if condition_on_doc(doc)]
    
    if texts is not None:
        for i in range(number_of_docs):
            doc = corpus[i]
            if condition_on_doc(doc) is False or doc == 'nan':
                texts[i] = ''
                corpus[i] = []
                

    print("{} docs removed".format(number_of_docs - len(corpus)))

    return (corpus, texts)

In [25]:
# Remove OOV words and documents with no words in model dictionary
def document_vector(word2vec_model, doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc if word in word2vec_model.vocab]
    return np.mean(word2vec_model[doc], axis=0)

In [26]:
def has_vector_representation(word2vec_model, doc):
    """check if at least one word of the document is in the
    word2vec dictionary"""
    return not all(word not in word2vec_model.vocab for word in doc)

In [29]:
t_corpus, manuals = filter_docs(t_corpus, manuals, lambda doc: has_vector_representation(model, doc))

0 docs removed


In [30]:
m_corpus, ocrs = filter_docs(m_corpus, ocrs, lambda doc: has_vector_representation(model, doc))
#m_corpus

0 docs removed


In [31]:
x1 =[]
for doc in t_corpus: #look up each doc in model
    #print("i: ", i)
    #i = i + 1
    if len(doc) == 0:
        x1.append(np.zeros((300,)))
    else:
        x1.append(document_vector(model, doc))

In [32]:
x2 =[]
i = 0
for doc in m_corpus: #look up each doc in model
    #print("i: ", i)
    #i = i + 1
    if len(doc) == 0:
        x2.append(np.zeros((300,)))
    else:
        x2.append(document_vector(model, doc))

In [33]:
cosine_similarity(x1[0].reshape(1,300), x2[0].reshape(1,300))

array([[ 0.19355321]], dtype=float32)

In [34]:
sim_list =[]
#i = 0
for (x1i, x2i) in zip(x1, x2):
    #print(x[i], y[i])
    x1i = x1i.reshape(1,300)
    x2i = x2i.reshape(1,300)
    sim = cosine_similarity(x1i, x2i)
    sim_list.append(sim[0,0])
    print(sim[0,0])
    #i = i + 1

0.193553
0.0
1.0
0.328193
0.149716
0.970492
0.761569
0.423941
0.984932
0.986207
0.298253
0.793486
0.624453
0.096267
0.173068
0.0
0.890463
0.841714
0.46813
0.0
0.754769
0.0
0.199203
0.690282
0.943864
0.168511
1.0
0.825322
0.524189
0.656407
0.0
1.0
0.978913
0.78532
0.987495
0.0466278
0.851722
0.216375
0.11087
0.646818
0.461531
1.0
0.948348
0.712156
0.721305
0.918384
0.332226
1.0
0.988905
0.0
0.0
0.76161
0.978913
0.985233
0.873303
0.258489
0.729841
0.865311
0.966607
0.81711
0.917696
0.772217
0.644702
0.990154
0.953024
0.768904
0.883063
0.969385
0.987001
0.979944
0.872394
0.380259
0.914013
0.801914
0.960069
0.0
0.99204
0.512558
0.0
0.926146
1.0
0.883063
0.935791
1.0
0.84058
1.0
0.758955
0.991987
0.699915
0.978906
0.860399
0.860399
0.881249
0.647007
0.984398
0.98718
0.802524
0.749554
0.978005
0.412551
0.600965
0.979819
0.813467
0.862722
0.915541
0.349743
0.931404
0.598852
0.363866
0.813467
0.813467
0.988782
0.979905
0.932914
0.431114
0.992684
0.73133
0.969761
0.977898
0.967094
0.763985
0.57

In [35]:
sim_list

[0.19355321,
 0.0,
 0.99999988,
 0.32819265,
 0.1497165,
 0.97049177,
 0.76156884,
 0.42394114,
 0.98493189,
 0.98620713,
 0.29825336,
 0.79348552,
 0.62445307,
 0.096267015,
 0.17306811,
 0.0,
 0.89046305,
 0.84171391,
 0.46813041,
 0.0,
 0.75476903,
 0.0,
 0.19920343,
 0.69028187,
 0.94386423,
 0.16851148,
 1.0,
 0.82532239,
 0.52418864,
 0.65640652,
 0.0,
 1.0,
 0.97891331,
 0.78532016,
 0.98749459,
 0.046627764,
 0.85172212,
 0.21637504,
 0.11087016,
 0.64681804,
 0.46153104,
 1.0000001,
 0.9483484,
 0.71215558,
 0.72130477,
 0.91838408,
 0.33222643,
 0.99999988,
 0.98890543,
 0.0,
 0.0,
 0.76160991,
 0.97891331,
 0.98523301,
 0.87330347,
 0.25848871,
 0.72984087,
 0.86531097,
 0.96660674,
 0.81711042,
 0.91769618,
 0.77221745,
 0.64470232,
 0.99015403,
 0.95302385,
 0.76890379,
 0.88306296,
 0.96938533,
 0.98700058,
 0.97994429,
 0.87239414,
 0.38025871,
 0.91401267,
 0.80191427,
 0.96006942,
 0.0,
 0.9920395,
 0.51255774,
 0.0,
 0.92614615,
 1.0000002,
 0.88306296,
 0.93579054,
 

In [36]:
FILE_READ_SIZE = 1024

In [37]:
sim_list.describe()

AttributeError: 'list' object has no attribute 'describe'

In [38]:
df_sim = pd.DataFrame(sim_list)

In [40]:
df_sim.describe()

,0
count,164.000000
mean,0.696123
std,0.322110
min,0.000000
25%,0.521281
50%,0.820617
75%,0.966756
max,1.000000
